# Perplexity LLM API
- [blog post introducing it](https://blog.perplexity.ai/blog/introducing-pplx-api)
- [api docs](https://docs.perplexity.ai/docs)
- [quickstart for chat completions](https://docs.perplexity.ai/reference/post_chat_completions)
- available models
    - codellama-34b-instruct, 16384
    - llama-2-70b-chat, 4096	
    - mistral-7b-instruct, 4096	
    - pplx-7b-chat, 8192	
    - pplx-70b-chat, 4096	
    - pplx-7b-online, 4096	
    - pplx-70b-online, 4096	

In [46]:
import openai
import os
import pandas as pd
import numpy as np
from pathlib import Path
from openai import OpenAI

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.document_loaders import TextLoader

In [47]:
## TODO not sourcing from bashrc, investigate why
#PERPLEXITY_API_KEY = os.environ.get('PERPLEXITY_API_KEY')

In [37]:
#PERPLEXITY_API_KEY=''

## Supported Models
- as of 2023-12-13
```
Model	Context Length	Model Type
codellama-34b-instruct	16384	Chat Completion
llama-2-70b-chat	4096	Chat Completion
mistral-7b-instruct	4096 [1]	Chat Completion
mixtral-8x7b-instruct	4096 [1]	Chat Completion
pplx-7b-chat	8192	Chat Completion
pplx-70b-chat	4096	Chat Completion
pplx-7b-online	4096	Chat Completion
pplx-70b-online	4096	Chat Completion
```

## Sample Code Structure they provide
* I updated the actual prompt though

In [48]:
client = OpenAI(base_url="https://api.perplexity.ai",
               api_key=PERPLEXITY_API_KEY)

In [49]:
messages = [
    {
        "role": "system",
        "content": (
            "You are an artificial intelligence assistant and you need to "
            "engage in a helpful, detailed, polite conversation with a user."
        ),
    },
    {
        "role": "user",
        "content": (
            "What are some simple tricks to improve my aim at darts?"
        ),
    },
]

# demo chat completion without streaming'

response = client.chat.completions.create(
    model="mistral-7b-instruct",
    messages=messages,
)
print(response)

ChatCompletion(id='ef6cc842-2221-463e-8cfa-762bf8d92906', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content="I'm glad you've asked about improving your aim in darts! Darts is a fun and challenging game that requires both precision and consistency. Here are some simple tricks that might help you improve your aim:\n\n1. Proper Grip: Hold the dart firmly with your dominant hand while keeping your other hand for stabilization. Make sure your fingers are curled around the dart with the tips pointing towards the front.\n2. Correct Stance: Stand with your feet shoulder-width apart, knees slightly bent, and your dominant foot slightly in front of the other. This helps maintain balance and control.\n3. Correct Posture: Keep your back straight, chest out, and head up. Make sure your throwing arm is relaxed and straight, and your non-dominant arm is extended for support.\n4. Focus on Your Target: Concentrate on the area of the dartboard you want to hit. It's imp

In [50]:
type(response)

openai.types.chat.chat_completion.ChatCompletion

In [55]:
response

ChatCompletion(id='ef6cc842-2221-463e-8cfa-762bf8d92906', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content="I'm glad you've asked about improving your aim in darts! Darts is a fun and challenging game that requires both precision and consistency. Here are some simple tricks that might help you improve your aim:\n\n1. Proper Grip: Hold the dart firmly with your dominant hand while keeping your other hand for stabilization. Make sure your fingers are curled around the dart with the tips pointing towards the front.\n2. Correct Stance: Stand with your feet shoulder-width apart, knees slightly bent, and your dominant foot slightly in front of the other. This helps maintain balance and control.\n3. Correct Posture: Keep your back straight, chest out, and head up. Make sure your throwing arm is relaxed and straight, and your non-dominant arm is extended for support.\n4. Focus on Your Target: Concentrate on the area of the dartboard you want to hit. It's imp

## Only print the response message

In [57]:
response.choices[0].message.content

"I'm glad you've asked about improving your aim in darts! Darts is a fun and challenging game that requires both precision and consistency. Here are some simple tricks that might help you improve your aim:\n\n1. Proper Grip: Hold the dart firmly with your dominant hand while keeping your other hand for stabilization. Make sure your fingers are curled around the dart with the tips pointing towards the front.\n2. Correct Stance: Stand with your feet shoulder-width apart, knees slightly bent, and your dominant foot slightly in front of the other. This helps maintain balance and control.\n3. Correct Posture: Keep your back straight, chest out, and head up. Make sure your throwing arm is relaxed and straight, and your non-dominant arm is extended for support.\n4. Focus on Your Target: Concentrate on the area of the dartboard you want to hit. It's important to keep your eyes fixed on the target throughout the throw.\n5. Proper Breathing: Take a deep breath before throwing the dart. Hold the 

## Quick Math on Current Pricing

In [58]:
pricing = pd.read_csv('pricing_for_perplexity_api.csv')

In [59]:
pricing

,model_parameter_count,per1m_input_tokens,per1m_output_tokens
0,7B,$0.07,$0.28
1,13B,$0.14,$0.56
2,34B,$0.35,$1.40
3,70B,$0.70,$2.80


In [60]:
pricing['per1m_input_tokens'] = pricing['per1m_input_tokens'].str.replace('$', '').astype(float)
pricing['per1m_output_tokens'] = pricing['per1m_output_tokens'].str.replace('$', '').astype(float)

In [61]:
pricing

,model_parameter_count,per1m_input_tokens,per1m_output_tokens
0,7B,0.07,0.28
1,13B,0.14,0.56
2,34B,0.35,1.40
3,70B,0.70,2.80


In [87]:
def cost_of_message(response,pricing=pricing):
    'return the cost of the individual message in USD'
    model_type = response.model.split('-')[-2].upper()
    input_tokens = response.usage.prompt_tokens
    output_tokens = response.usage.completion_tokens

    input_rate = pricing[pricing.model_parameter_count == model_type].per1m_input_tokens
    output_rate = pricing[pricing.model_parameter_count == model_type].per1m_output_tokens

    input_cost = input_tokens * input_rate / 1_000_000
    output_cost = output_tokens * output_rate / 1_000_000

    cost = input_cost + output_cost
    return cost

In [88]:
cost_of_message(response=response,
                pricing=pricing)

0    0.000114
dtype: float64

In [89]:
system_prompt = {
    'role': 'system',
     'content': 'You are an artificial intelligence assistant and you need to engage in a helpful, detailed, polite conversation with a user.'
    }

## Function to get answers

In [ ]:
client = OpenAI(base_url="https://api.perplexity.ai",
               api_key=PERPLEXITY_API_KEY)

In [102]:
def ask_ppl(question,
            system_prompt=system_prompt,
            max_tokens=2048, 
            model="mistral-7b-instruct",
           client=client):

    user_prompt = {
        "role": "user",
        "content": (
           question
        ),
    }

    message= [
            system_prompt,
            user_prompt
        ]
    
    response = client.chat.completions.create(
                                                model=model,
                                                messages=message,
                                                max_tokens=max_tokens
                                            )

    return response.choices[0].message.content

## Test Function

In [103]:
max_tokens = 2048
model='mistral-7b-instruct'

In [104]:
models = [
    'codellama-34b-instruct',
    'llama-2-70b-chat',
    'mistral-7b-instruct',
    'pplx-7b-chat',
    'pplx-70b-chat',
]

In [105]:
model = models[1]

## Compare Model Answers

In [106]:
question = 'who in your estimation are the most influential computer scientists of the 21st century?'

In [ ]:
model_answers = {}

for model in models:
    print(f'Asking {model}')
    answer = ask_ppl(question=question, model=model, client=client)
    model_answers[model] = answer

In [110]:
model_answers.keys()

dict_keys(['codellama-34b-instruct'])

In [111]:
model_answers['codellama-34b-instruct']

'There have been many influential computer scientists in the 21st century who have made significant contributions to the field. Some of the most influential computer scientists of the 21st century include:\n\n1. Alan Turing: Known as the father of computer science and artificial intelligence, Alan Turing made fundamental contributions to the fields of computer science, mathematics, and philosophy.\n2. Tim Berners-Lee: Tim Berners-Lee is credited with inventing the World Wide Web and is considered the father of the internet. He is also a noted computer scientist and engineer.\n3. Marvin Minsky: Marvin Minsky was a influential computer scientist and cognitive scientist who made significant contributions to the fields of artificial intelligence, neural networks, and cognitive science.\n4. Andrew Ng: Andrew Ng is a computer scientist and entrepreneur who has made significant contributions to the fields of artificial intelligence and machine learning. He is also the founder of Coursera and 

# Use as a RAG

## Embed Articles from The Last Psychiatrist

In [112]:
# use tiny model for embeddings
model_id = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_id)

In [113]:
# Load text data from a file using TextLoader
loader = TextLoader('data/who_can_know_how_much_randi_zu.txt')
documents = loader.load()

In [114]:
# format it for processing by embedding model
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
text_chunks = text_splitter.split_documents(documents)

In [115]:
vectorstore_of_docs = FAISS.from_documents(text_chunks, embedding=embeddings)

## Function to prep docs for encoding

In [116]:
def load_text_files_for_encoding(file_names, dir):
    documents = []
    for text_file in file_names:
        loader = TextLoader(str(directory) + '/' + text_file)
        documents.extend(loader.load())
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
    chunked_documents = text_splitter.split_documents(documents)

    return chunked_documents

## Encoding a bunch of documents

In [117]:
directory = Path('/home/nick/Documents/repos/llm_playground/data')

file_names = [file.name for file in directory.glob('*.txt')]

chunked_documents = load_text_files_for_encoding(file_names, directory)

In [118]:
vectorstore_of_docs = FAISS.from_documents(chunked_documents, embedding=embeddings)

## Test the embeddings

In [119]:
related_text = vectorstore_of_docs.similarity_search('narcissism',top_k=3)

In [120]:
related_text

[Document(page_content="who like her-- the people she has to settle for-- are... not great.Genetics took care of her body but the upbringing affected her vision: the childhood of never good enough filters her present reality, obscures it, she can't see what is plain to everyone else, e.g. she's beautiful. So the process is to uncover the reasons why her view of reality is distorted and help her realign with reality. Use insight to strengthen her damaged ego, or, if you want a ten step approach, block automatic thoughts. In short, to understand that she is good, that men do find her attractive, not just the brazen ones, not just jerks.IV.If you think of narcissism as grandiosity you miss the nuances, e.g. in her case the problem is narcissism without any grandiosity: she is so consumed with her identity (as not pretty) that she is not able to read, to empathize with, other people's feelings. She doesn't care to try because it conflicts with how she sees herself. Ergo: Giorgio Armani was

## Ask Question with Context

In [121]:
def ask_ppl(question,
            system_prompt=system_prompt,
            max_tokens=2048, 
            model="mistral-7b-instruct",
            client=client,
           context=False,
           top_k=5,
           vectorstore=False):
    'Ask the perplexity api a question, with or without context, using whichever model you prefer'

    if context == True:
        # find relevant text and add it to question
        related_text = vectorstore.similarity_search(question,top_k)
        context = ' / '.join([related_text[i].page_content for i in range(top_k)])
        files_used = ','.join(set([related_text[i].metadata['source'] for i in range(top_k)]))
        context_string = f'{context} || from the files {files_used}'
        # update question string
        question =  f'''Use the following pieces of context to answer the question at the end. 
            Try to make the response brief and only answer the portion after Question:
            Do not refer to the text from this prompt outside of the context.
            Structure your answers as well composed english sentences and do not include '\n'
            
            {context_string}
            
            Question: {question}
            Answer: '''

    user_prompt = {
        "role": "user",
        "content": (
           question
        ),
    }
    
    message= [
            system_prompt,
            user_prompt
        ]
    
    response=client.chat.completions.create(
                                                    model=model,
                                                    messages=message,
                                                    max_tokens=max_tokens
                                                )

    return response.choices[0].message.content

In [122]:
test_query = 'Why is Randi Zuckerberg well known?'

In [123]:
ask_ppl(test_query)

'Randi Zuckerberg is a well-known entrepreneur, technology innovator, and media executive. She is the older sister of Mark Zuckerberg, the founder and CEO of Facebook. Randi joined Facebook in its early days, first working as a marketing manager and later becoming the company\'s director of market development. While at Facebook, she developed the idea for a live-streaming product, which later became Facebook Live.\n\nAfter leaving Facebook in 2011, Randi founded RtoZ Media, a production company that produces videos for digital and television platforms. She also hosts a daily business news program on SiriusXM radio called "Dot Complicated with Randi Zuckerberg." Additionally, she\'s an accomplished singer and has released several albums.\n\nRandi is a frequent speaker at events focused on technology, media, and entrepreneurship. She often shares her insights and experiences on various stages and in interviews, making her a prominent figure in these industries. Her unique background, tal

In [124]:
ask_ppl(question=test_query,
            system_prompt=system_prompt,
            max_tokens=2048, 
            model="mistral-7b-instruct",
           context=True,
           top_k=5,
       vectorstore=vectorstore_of_docs)

'Randi Zuckerberg is well known due to her relation to Mark Zuckerberg, her brother, and her work as the CEO of Zuckerberg Media. She has also given numerous interviews, written a book about social media, and has spoken at events like TEDx and the World Economic Forum. However, some criticize her fame as being largely due to nepotism'

In [128]:
ask_ppl(question=test_query,
            system_prompt=system_prompt,
            max_tokens=2048, 
            model='mixtral-8x7b-instruct',
           context=True,
           top_k=5,
       vectorstore=vectorstore_of_docs)

'Randi Zuckerberg is well known because she is the CEO of Zuckerberg Media and the sister of Mark Zuckerberg, the co-founder of Facebook. She has also written a book about social media, given many interviews, and attended events like the World Economic Forum, which has contributed to her fame.'

In [129]:
test_query_2 = "what is narcissism, according to the Last Psychiatrist?"

In [130]:
ask_ppl(question=test_query_2,
            system_prompt=system_prompt,
            max_tokens=2048, 
            model="mistral-7b-instruct",
           context=True,
           top_k=5,
       vectorstore=vectorstore_of_docs)

'In the context provided by "The Last Psychiatrist," narcissism is a personality disorder characterized by identity preservation and an inability to see things from other people\'s perspectives. The author emphasizes the difficulty in treating narcissism due to these traits, which make admitting one\'s problem and acknowledging others\' perspectives nearly impossible. Additionally, they suggest that a narcissistic person may never have truly known themselves due to being shielded from real experiences and challenges in childhood'

In [131]:
ask_ppl(question=test_query_2,
            system_prompt=system_prompt,
            max_tokens=2048, 
            model='mixtral-8x7b-instruct',
           context=True,
           top_k=5,
       vectorstore=vectorstore_of_docs)

"According to The Last Psychiatrist, narcissism is a personality disorder characterized by identity preservation and an inability to see things from others' perspectives. It's not about self-love, but rather a lack of ability to love others, which eventually leads to self-absorption. The narcissist's identity is tied to maintaining their self-image and avoiding guilt or objective reality that doesn't comply with their mirroring demands. Narcissism develops when a child is not allowed to know themselves through real-life experiences, but instead, is sheltered and surrounded by manipulated situations."

In [132]:
test_query_3 = "if everything is a defense against actually changing, what does Alone think people should do?"

In [133]:
ask_ppl(question=test_query_3,
            system_prompt=system_prompt,
            max_tokens=2048, 
            model="mistral-7b-instruct",
           context=True,
           top_k=5,
       vectorstore=vectorstore_of_docs)

'Alone does not explicitly suggest what people should do in the context provided. However, the text implies that Alone encourages thinking critically about the motivations behind change and questioning the roles we play in perpetuating the status quo. Additionally, the text suggests that meaningful change may not be achievable through relying on external sources like media or activist groups. Instead, one might consider taking personal actions to effect change in their own lives and communities'

In [136]:
ask_ppl(question=test_query_3,
            system_prompt=system_prompt,
            max_tokens=2048, 
            model='mixtral-8x7b-instruct',
           context=True,
           top_k=5,
       vectorstore=vectorstore_of_docs)

"Alone suggests that people should strive for critical thinking and independence of thought, in order to see through the system's attempts to control them. This means questioning the information they receive, seeking out diverse perspectives, and not blindly adhering to any one ideology or viewpoint. By doing so, individuals can better understand the world around them and make informed decisions, rather than being manipulated by the status quo."